In [ ]:
import squigglepy as sq
import numpy as np
import pandas as pd
from squigglepy.numbers import K, M, B
from chip_estimates_utils import (
    normalize_shares,
    compute_h100_equivalents,
    export_quarterly_by_version,
    print_cumulative_summary,
    estimate_chip_sales,
    estimate_cumulative_chip_sales,
)

sq.set_seed(42)
np.random.seed(42)
N_SAMPLES = 1000

In [2]:
# ======================
# Fake sample data
# ======================

# Chip specs with prices (uncertain)
CHIP_SPECS = {
    'alpha':   {'price': sq.to(800, 1200)},
    'beta':    {'price': sq.to(1500, 2500)},
    'gamma':   {'price': sq.to(3000, 5000)},
    'delta':   {'price': sq.to(8000, 15000)},
}

# Quarterly revenue estimates (in billions)
REVENUE = {
    'Q1_2023': sq.to(0.8, 1.2) * B,
    'Q2_2023': sq.to(1.0, 1.5) * B,
    'Q3_2023': sq.to(1.2, 1.8) * B,
    'Q4_2023': sq.to(1.5, 2.2) * B,
    'Q1_2024': sq.to(1.8, 2.8) * B,
    'Q2_2024': sq.to(2.2, 3.5) * B,
    'Q3_2024': sq.to(2.8, 4.2) * B,
    'Q4_2024': sq.to(3.5, 5.0) * B,
}

# Production mix by quarter (will be normalized)
PROD_MIX = {
    'Q1_2023': {'alpha': sq.to(0.70, 0.85), 'beta': sq.to(0.15, 0.30)},
    'Q2_2023': {'alpha': sq.to(0.50, 0.70), 'beta': sq.to(0.25, 0.40), 'gamma': sq.to(0.01, 0.10)},
    'Q3_2023': {'alpha': sq.to(0.30, 0.50), 'beta': sq.to(0.30, 0.50), 'gamma': sq.to(0.10, 0.25)},
    'Q4_2023': {'alpha': sq.to(0.20, 0.35), 'beta': sq.to(0.30, 0.45), 'gamma': sq.to(0.20, 0.35), 'delta': sq.to(0.01, 0.08)},
    'Q1_2024': {'alpha': sq.to(0.10, 0.25), 'beta': sq.to(0.25, 0.40), 'gamma': sq.to(0.25, 0.40), 'delta': sq.to(0.08, 0.18)},
    'Q2_2024': {'alpha': sq.to(0.05, 0.15), 'beta': sq.to(0.20, 0.35), 'gamma': sq.to(0.30, 0.45), 'delta': sq.to(0.15, 0.30)},
    'Q3_2024': {'alpha': sq.to(0.02, 0.10), 'beta': sq.to(0.15, 0.30), 'gamma': sq.to(0.30, 0.45), 'delta': sq.to(0.25, 0.40)},
    'Q4_2024': {'alpha': sq.to(0.01, 0.08), 'beta': sq.to(0.10, 0.25), 'gamma': sq.to(0.30, 0.45), 'delta': sq.to(0.30, 0.50)},
}

In [3]:
# Define sampling functions which get passed into estimate_chip_sales
def sample_revenue(quarter):
    return REVENUE[quarter] @ 1

def sample_shares(quarter):
    mix = PROD_MIX[quarter]
    raw_shares = {chip_type: dist @ 1 for chip_type, dist in mix.items()}
    return normalize_shares(raw_shares)

def sample_price(quarter, chip_type):
    return CHIP_SPECS[chip_type]['price'] @ 1

## Quarter correlations model

In [ ]:
# ======================
# Price data by year (fake)
# ======================

YEARLY_PRICES = {
    'alpha': {2023: sq.to(900, 1100),  2024: sq.to(650, 850)},
    'beta':  {2023: sq.to(1800, 2400), 2024: sq.to(1400, 1900)},
    'gamma': {2023: sq.to(3500, 4500), 2024: sq.to(2800, 3800)},
    'delta': {2023: sq.to(10000, 14000), 2024: sq.to(7500, 11000)},
}

# Revenue estimation bias
REVENUE_BIAS = sq.to(0.85, 1.15)

# Helper to get year from quarter
def get_year(quarter):
    return int(quarter.split('_')[1])

# Compute geometric mean of a distribution (for deflation ratios)
def geo_mean(dist, n=1000):
    samples = dist @ n
    return np.exp(np.mean(np.log(samples)))

# Build deflation factors from yearly price data
def get_deflation_factor(quarter, chip):
    """Compute deflation as ratio of current year's price to first year's price."""
    year = get_year(quarter)
    years = sorted(YEARLY_PRICES[chip].keys())
    first_year = years[0]
    
    if year == first_year:
        return 1.0
    
    base_mean = geo_mean(YEARLY_PRICES[chip][first_year])
    current_mean = geo_mean(YEARLY_PRICES[chip][year])
    return current_mean / base_mean

# Sample base price (from first year's distribution)
def sample_base_price(chip):
    """Sample from first year's price distribution."""
    first_year = min(YEARLY_PRICES[chip].keys())
    return YEARLY_PRICES[chip][first_year] @ 1

# Print deflation factors for reference
print("Deflation factors (ratio of geo means to first year):")
for chip in YEARLY_PRICES:
    factors = {year: get_deflation_factor(f"Q1_{year}", chip) 
               for year in sorted(YEARLY_PRICES[chip].keys())}
    print(f"  {chip}: {factors}")

In [ ]:
# estimate_cumulative_chip_sales is now in chip_estimates_utils.py
# 
# Usage:
#   from chip_estimates_utils import estimate_cumulative_chip_sales
#
#   cumulative = estimate_cumulative_chip_sales(
#       quarters=list(REVENUE.keys()),
#       chip_types=list(CHIP_SPECS.keys()),
#       sample_revenue=sample_revenue,
#       sample_shares=sample_shares,
#       sample_base_price=sample_base_price,  # fn(chip) -> float, samples BASE price
#       get_deflation_factor=get_deflation_factor,  # fn(quarter, chip) -> float, or None
#       revenue_bias_dist=REVENUE_BIAS,  # squigglepy distribution, or None
#       n_samples=N_SAMPLES
#   )
#
# Key differences from estimate_chip_sales:
#   - sample_base_price(chip) instead of sample_price(quarter, chip)
#     Returns the introduction/base price for a chip, called once per chip
#   - get_deflation_factor(quarter, chip) scales base price for each quarter
#     Should return 1.0 for first quarter, <1.0 for later quarters
#   - revenue_bias_dist: optional systematic bias applied to all quarters
#   - Returns cumulative totals {chip: np.array} instead of per-quarter results
#
# See chip_estimates_utils.py for full docstring and implementation.

from chip_estimates_utils import estimate_cumulative_chip_sales

In [ ]:
# Uncorrelated version for comparison
def sample_price_with_deflation(quarter, chip):
    """Sample price from that year's distribution (uncorrelated - resamples each time)."""
    year = get_year(quarter)
    return YEARLY_PRICES[chip][year] @ 1

def sample_revenue_with_bias(quarter):
    """Sample revenue with bias (but bias is uncorrelated across quarters)."""
    return (REVENUE[quarter] @ 1) * (REVENUE_BIAS @ 1)

uncorr_results = estimate_chip_sales(
    quarters=list(REVENUE.keys()),
    versions=list(CHIP_SPECS.keys()),
    sample_revenue=sample_revenue_with_bias,
    sample_shares=sample_shares,
    sample_price=sample_price_with_deflation,
    n_samples=N_SAMPLES
)

# Aggregate to cumulative
cumulative_uncorr = {chip: np.zeros(N_SAMPLES) for chip in CHIP_SPECS}
for quarter in uncorr_results:
    for chip in CHIP_SPECS:
        cumulative_uncorr[chip] += np.array(uncorr_results[quarter][chip])

print_cumulative_summary(cumulative_uncorr, CHIP_SPECS, "Cumulative Chip Production (uncorrelated)")

In [ ]:
# Run the full correlation model
cumulative_full = estimate_cumulative_chip_sales(
    quarters=list(REVENUE.keys()),
    chip_types=list(CHIP_SPECS.keys()),
    sample_revenue=sample_revenue,
    sample_shares=sample_shares,
    sample_base_price=sample_base_price,
    get_deflation_factor=get_deflation_factor,
    revenue_bias_dist=REVENUE_BIAS,
    n_samples=N_SAMPLES
)

print_cumulative_summary(cumulative_full, CHIP_SPECS, "Cumulative Chip Production (full correlation)")

# Side-by-side comparison of variance
print("\n\nVariance comparison (correlated should have wider p5-p95 range):")
for chip in CHIP_SPECS:
    uncorr_range = np.percentile(cumulative_uncorr[chip], 95) - np.percentile(cumulative_uncorr[chip], 5)
    corr_range = np.percentile(cumulative_full[chip], 95) - np.percentile(cumulative_full[chip], 5)
    print(f"  {chip}: uncorr={uncorr_range:,.0f}, corr={corr_range:,.0f}, ratio={corr_range/uncorr_range:.2f}x")

### How the correlation model works

Similar interface to `estimate_chip_sales`, but with `sample_base_price(chip)` instead of `sample_price(quarter, chip)`.

**Correlated parameters:**
1. **Base prices** - `sample_base_price(chip)` sampled once per chip, then scaled by `get_deflation_factor(quarter, chip)` each quarter
2. **Revenue bias** (optional) - sampled once, applied to all quarters

**Uncorrelated parameters** (sampled fresh each quarter):
- Base revenue (before bias)
- Production mix shares

The deflation function computes price ratios from `YEARLY_PRICES` - all the year/deflation logic lives outside the estimation function.